In [54]:
import torch
from PIL import Image
from tools import (ego_to_cam, get_only_in_img_mask, denormalize_img, normalize_img,
                    SimpleLoss, get_val_info, add_ego, gen_dx_bx,img_transform,
                    get_nusc_maps, plot_nusc_map, generate_video_from_imgs)
from models import compile_model

In [44]:
# Add paths
dataroot = '/home/navya/data/LSS/LSSCAM'
map_folder = '/home/navya/data/nuscenes/trainval/'
modelf = '/home/navya/project/lift-splat-shoot/model70000.pt'
imgname = '/home/navya/project/lift-splat-shoot/input.jpg'

In [62]:
import numpy as np
def sample_augmentation():
        H, W = data_aug_conf['H'], data_aug_conf['W']
        fH, fW = data_aug_conf['final_dim']
        # if self.is_train:
        #     resize = np.random.uniform(*self.data_aug_conf['resize_lim'])
        #     resize_dims = (int(W*resize), int(H*resize))
        #     newW, newH = resize_dims
        #     crop_h = int((1 - np.random.uniform(*self.data_aug_conf['bot_pct_lim']))*newH) - fH
        #     crop_w = int(np.random.uniform(0, max(0, newW - fW)))
        #     crop = (crop_w, crop_h, crop_w + fW, crop_h + fH)
        #     flip = False
        #     if self.data_aug_conf['rand_flip'] and np.random.choice([0, 1]):
        #         flip = True
        #     rotate = np.random.uniform(*self.data_aug_conf['rot_lim'])
        # else:
        resize = max(fH/H, fW/W)
        resize_dims = (int(W*resize), int(H*resize))
        newW, newH = resize_dims
        crop_h = int((1 - np.mean(data_aug_conf['bot_pct_lim']))*newH) - fH
        crop_w = int(max(0, newW - fW) / 2)
        crop = (crop_w, crop_h, crop_w + fW, crop_h + fH)
        flip = False
        rotate = 0
        return resize, resize_dims, crop, flip, rotate

In [47]:
# Initialization
gpuid=0
viz_train=False
video_output=True
max_frames=-1
channel=1

H=900
W=1600
resize_lim=(0.193, 0.225)
final_dim=(128, 352)
bot_pct_lim=(0.0, 0.22)
rot_lim=(-5.4, 5.4)
rand_flip=True

xbound=[-50.0, 50.0, 0.5]
ybound=[-50.0, 50.0, 0.5]
zbound=[-10.0, 10.0, 20.0]
dbound=[4.0, 45.0, 1.0]

bsz=1
nworkers=0

In [48]:
grid_conf = {
    'xbound': xbound,
    'ybound': ybound,
    'zbound': zbound,
    'dbound': dbound,
}

cams = ['CAM_FRONT']

data_aug_conf = {
    'resize_lim': resize_lim,
    'final_dim': final_dim,
    'rot_lim': rot_lim,
    'H': H, 'W': W,
    'rand_flip': rand_flip,
    'bot_pct_lim': bot_pct_lim,
    'cams': cams,
    'Ncams': 1,
}

In [49]:
nusc_maps = get_nusc_maps(map_folder)

device = torch.device('cpu') if gpuid < 0 else torch.device(f'cuda:{gpuid}')

model = compile_model(grid_conf, data_aug_conf, outC=3)
print('loading', modelf)
model.load_state_dict(torch.load(modelf))
model.to(device)

dx, bx, _ = gen_dx_bx(grid_conf['xbound'], grid_conf['ybound'], grid_conf['zbound'])
dx, bx = dx[:2].numpy(), bx[:2].numpy()

Loaded pretrained weights for efficientnet-b0
loading /home/navya/project/lift-splat-shoot/model70000.pt


In [50]:
val = 0.01
fH, fW = final_dim
model.eval()
counter = 0

In [ ]:
imgs = []
img = Image.open(imgname)
imgs.append(normalize_img(img))
imgs = torch.stack(imgs)

In [60]:
rots = torch.tensor([[ 0.0103,  0.0084,  0.9999],
                    [-0.9999,  0.0123,  0.0102],
                    [-0.0122, -0.9999,  0.0086]])

trans = torch.tensor([ 1.7220,  0.0048,  1.4949])
    
intrins = torch.tensor([[1.2528e+03, 0.0000e+00, 8.2659e+02],
                    [0.0000e+00, 1.2528e+03, 4.6998e+02],
                    [0.0000e+00, 0.0000e+00, 1.0000e+00]])
    
post_rot = [[1., 0., 0.],
            [0., 1., 0.],
            [0., 0., 1.]]

post_tran = [0., 0., 0.]

In [63]:
# augmentation (resize, crop, horizontal flip, rotate)
resize, resize_dims, crop, flip, rotate = sample_augmentation()
img, post_rot2, post_tran2 = img_transform(img, post_rot, post_tran,
                                            resize=resize,
                                            resize_dims=resize_dims,
                                            crop=crop,
                                            flip=flip,
                                        rotate=rotate,
                                            )

# for convenience, make augmentation matrices 3x3
post_tran = torch.zeros(3)
post_rot = torch.eye(3)
post_tran[:2] = post_tran2
post_rot[:2, :2] = post_rot2

TypeError: can't multiply sequence by non-int of type 'float'

In [53]:
with torch.no_grad():
    out = model(imgs.to(device),
        rots.to(device),
        trans.to(device),
        intrins.to(device),
        post_rots.to(device),
        post_trans.to(device),
        )
    out = out.sigmoid().cpu()

    plt.imshow(out[.channel])

ValueError: not enough values to unpack (expected 3, got 1)